In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

MessageError: ignored

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab import drive
import matplotlib.pyplot as plt
import joblib
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import plotly.express as px

In [ ]:
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

In [ ]:
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.metrics import classification_report
from skimage.feature import local_binary_pattern,hog
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import classification_report

In [ ]:
print (cv2 .__version__)

4.1.2


#Load Data

In [ ]:
y_train = np.load("/content/gdrive/MyDrive/ISS-Project/NumpyArrays/train_y.npy")
y_val = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/val_y.npy')

features_train_pca = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/features_train_pca.npy')
features_val_pca = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/features_val_pca.npy')
features_test_pca = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/features_test_pca.npy')

features_train_lda = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/features_train_lda.npy')
features_val_lda = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/features_val_lda.npy')
features_test_lda = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/features_test_lda.npy')

lda_pca_train = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/lda_pca_train.npy')
lda_pca_val = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/lda_pca_val.npy')
lda_pca_test=np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/lda_pca_test.npy')

In [ ]:
train_imgs = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/train_imgs_color.npy')
train_y = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/train_y_color.npy')

val_imgs = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/val_imgs_color.npy')
val_y = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/val_y_color.npy')

X = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/X_color.npy')
y = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/y_color.npy')

X_test = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/X_test_color.npy')
y_test = np.load('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/y_test_color.npy')

In [ ]:
def train_model(train_x, train_y, model_name='DT',data_type='PCA', validation=None):
  # """
   # Possible model names: ['SVM', 'ADA','KNN','BAG', 'XGB', 'DT']
   # default = 'DT'

    #validation: (val_x, val_y) tupple for validation accuracy score.

   # """
   model = None
   if model_name == 'SVM' and data_type == 'PCA':
     model = svm.SVC(C=100, kernel='rbf', probability=True)
   if model_name == 'SVM' and data_type == 'LDA':
     model = svm.SVC(C=0.1, kernel='rbf', probability=True)
   if model_name == 'SVM' and data_type == 'LDA-PCA':
     model = svm.SVC(C=1, kernel='rbf', probability=True)
   if model_name == 'KNN' and data_type == 'PCA':
     model = KNeighborsClassifier(n_neighbors=5)
   if model_name == 'KNN' and data_type == 'LDA':
     model = KNeighborsClassifier(n_neighbors=51)
   if model_name == 'KNN' and data_type == 'LDA-PCA':
     model = KNeighborsClassifier(n_neighbors=5)
   if model_name == 'ADA' and data_type == 'PCA':
     model = AdaBoostClassifier(n_estimators=200)
   if model_name == 'ADA' and data_type == 'LDA':
     model = AdaBoostClassifier(n_estimators=200)
   if model_name == 'ADA' and data_type == 'LDA-PCA':
     model = AdaBoostClassifier(n_estimators=50)
   if model_name == 'BAG' and data_type == 'PCA':
     model = BaggingClassifier(n_estimators=40)
   if model_name == 'BAG' and data_type == 'LDA':
     model = BaggingClassifier(n_estimators=40)
   if model_name == 'BAG' and data_type == 'LDA-PCA':
     model = BaggingClassifier(n_estimators=20)
   if model_name == 'DT' and data_type == 'PCA':
     model = DecisionTreeClassifier(criterion='entropy', max_depth=20)
   if model_name == 'DT' and data_type == 'LDA':
     model = DecisionTreeClassifier(criterion='entropy', max_depth=15)
   if model_name == 'DT' and data_type == 'LDA-PCA':
     model = DecisionTreeClassifier(criterion='entropy', max_depth=15)

   model.fit(train_x, train_y)

   if validation is not None:
     target_names = ['class 0', 'class 1', 'class 2', 'class 3', 'class 4', 'class 5', 'class 6', 'class 7','class 8', 'class 9']
     y_hat = model.predict(validation[0])
     class_report = classification_report(validation[1], y_hat, target_names=target_names, output_dict=True)
     print('Precision =', class_report['macro avg']['precision'])
     print('Recall = ', class_report['macro avg']['recall'])
     print('F1-score =', class_report['macro avg']['f1-score'])
     print('Accuracy =', class_report['accuracy'])

   return model

# Grid Search

### Model #1 Adaboost

In [ ]:
def apply_grid_search_ADA(features_train, train_y):
    grid_params = {'n_estimators': [50, 100, 200]}
    model = AdaBoostClassifier()
    #cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    grid_object = GridSearchCV(estimator=model, param_grid=grid_params, scoring='accuracy', n_jobs=-1, refit=True,
                               verbose=3)
    grid_object.fit(features_train, train_y)
    best_accuracy = grid_object.best_score_
    best_parameters = grid_object.best_params_
    print("Best Accuracy: {:.2f} %".format(best_accuracy * 100))
    print("Best Parameters:", best_parameters)
    return grid_object

In [ ]:
# Adaboost PCA
ada_pca_model = apply_grid_search_ADA(features_train_pca, y_train)
pickle.dump(ada_pca_model, open('/content/gdrive/MyDrive/ISS-Project/Models/ada_pca_model.pkl', 'wb'))
#ada_pca_model = pickle.load(open('/content/gdrive/MyDrive/ISS-Project/Models/ada_pca_model.pkl', 'rb'))
print(ada_pca_model)

In [ ]:
# Adaboost LDA
ada_lda_model = apply_grid_search_ADA(features_train_lda, y_train)
pickle.dump(ada_lda_model, open('/content/gdrive/MyDrive/ISS-Project/Models/ada_lda_model.pkl', 'wb'))
#ada_lda_model = pickle.load(open('/content/gdrive/MyDrive/ISS-Project/Models/ada_lda_model.pkl', 'rb'))
print(ada_lda_model)


In [ ]:
# Adaboost PCA-LDA
ada_lda_pca_model = apply_grid_search_ADA(lda_pca_train, y_train)
pickle.dump(ada_lda_pca_model, open('/content/gdrive/MyDrive/ISS-Project/Models/ada_lda_pca_model.pkl', 'wb'))
#ada_lda_pca_model = pickle.load(open('/content/gdrive/MyDrive/ISS-Project/Models/ada_lda_pca_model.pkl', 'rb'))
print(ada_lda_pca_model)


Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   24.3s finished


Best Accuracy: 68.61 %
Best Parameters: {'n_estimators': 50}
GridSearchCV(cv=None, error_score=nan,
             estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                          base_estimator=None,
                                          learning_rate=1.0, n_estimators=50,
                                          random_state=None),
             iid='deprecated', n_jobs=-1,
             param_grid={'n_estimators': [50, 100, 200]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=3)


#### Train Best Model ( Adaboost )


##### PCA
1.   Best Parameters: {'n_estimators': 200}
2.   Best Accuracy: 60.86 %
3.   Best Accuracy Testing Data: 20 %

##### LDA
1.   Best Parameters: {'n_estimators': 200}
2.   Best Accuracy: 93.24 %
3.   Best Accuracy Testing Data: 67 %

##### LDA - PCA
1.   Best Parameters: {'n_estimators': 50}
2.   Best Accuracy: 68.61 %
3.   Best Accuracy Testing Data: 16 %





In [ ]:
comp_model = train_model(features_train_pca, y_train, model_name='ADA',data_type='PCA')
pickle.dump(comp_model, open('/content/gdrive/MyDrive/ISS-Project/Models/final_ada_pca_model.pkl', 'wb'))
y_hat = comp_model.predict(features_test_pca)
np.save('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/final_ada_pca_predictions.npy', y_hat)
acc = metrics.accuracy_score(y_test, y_hat)
print(acc)

0.20958751393534003


In [ ]:
comp_model = train_model(features_train_lda, y_train, model_name='ADA',data_type='LDA')
pickle.dump(comp_model, open('/content/gdrive/MyDrive/ISS-Project/Models/final_ada_lda_model.pkl', 'wb'))
y_hat = comp_model.predict(features_test_lda)
np.save('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/final_ada_lda_predictions.npy', y_hat)
acc = metrics.accuracy_score(y_test, y_hat)
print(acc)

0.6767001114827201


In [ ]:
comp_model = train_model(lda_pca_train, y_train, model_name='ADA',data_type='LDA-PCA')
pickle.dump(comp_model, open('/content/gdrive/MyDrive/ISS-Project/Models/final_ada_lda_pca_model.pkl', 'wb'))
y_hat = comp_model.predict(lda_pca_test)
np.save('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/final_ada_lda_pca_predictions.npy', y_hat)
acc = metrics.accuracy_score(y_test, y_hat)
print(acc)

0.16387959866220736


### Model #2 SVM

In [ ]:
def apply_grid_search_SVC(features_train, train_y):
    param_grid = {'C': [0.1, 1, 10, 100],
                  'kernel': ['rbf']}
    model = SVC(probability=True)
    grid_object = GridSearchCV(estimator=model, param_grid=param_grid,scoring='accuracy', n_jobs=-1, refit=True,
                               verbose=3)
    grid_object.fit(features_train, train_y)
    best_accuracy = grid_object.best_score_
    best_parameters = grid_object.best_params_
    print("Best Accuracy: {:.2f} %".format(best_accuracy * 100))
    print("Best Parameters:", best_parameters)
    return grid_object


In [ ]:
# SVM -PCA
svm_pca_model = apply_grid_search_SVC(features_train_pca, y_train)
pickle.dump(svm_pca_model, open('/content/gdrive/MyDrive/ISS-Project/Models/svm_pca_model.pkl', 'wb'))
#svm_pca_model = pickle.load(open('/content/gdrive/MyDrive/ISS-Project/Models/svm_pca_model.pkl', 'rb'))
print(svm_pca_model)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 43.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 43.3min finished


Best Accuracy: 99.76 %
Best Parameters: {'C': 100, 'kernel': 'rbf'}
GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=3)


In [ ]:
# SVM LDA
svm_lda_model = apply_grid_search_SVC(features_train_lda, y_train)
pickle.dump(svm_lda_model, open('/content/gdrive/MyDrive/ISS-Project/Models/svm_lda_model.pkl', 'wb'))
#svm_lda_model = pickle.load(open('/content/gdrive/MyDrive/ISS-Project/Models/svm_lda_model.pkl', 'rb'))
print(svm_lda_model)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.7min finished


Best Accuracy: 96.87 %
Best Parameters: {'C': 0.1, 'kernel': 'rbf'}
GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=3)


In [ ]:
# SVM PCA-LDA
svm_lda_pca_model = apply_grid_search_SVC(lda_pca_train, y_train)
pickle.dump(svm_lda_pca_model, open('/content/gdrive/MyDrive/ISS-Project/Models/svm_lda_pca_model.pkl', 'wb'))
#svm_lda_pca_model = pickle.load(open('/content/gdrive/MyDrive/ISS-Project/Models/svm_lda_pca_model.pkl', 'rb'))
print(svm_lda_pca_model)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   23.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   23.5s finished


Best Accuracy: 98.43 %
Best Parameters: {'C': 1, 'kernel': 'rbf'}
GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=3)


#### Train Best Model ( SVM )

##### PCA

1.   Best Parameters: {'C': 100, 'kernel': 'rbf'}
2.   Best Accuracy: 99.76 %
3.   Best Accuracy Testing Data: 18 %

##### LDA

1.   Best Parameters: {'C': 0.1, 'kernel': 'rbf'}
2.   Best Accuracy: 96.87 %
3.   Best Accuracy Testing Data: 77 %

##### LDA - PCA

1.   Best Parameters: {'C': 1, 'kernel': 'rbf'}
2.   Best Accuracy: 98.43 %
3.   Best Accuracy Testing Data: 19 %






In [ ]:
comp_model = train_model(features_train_pca, y_train, model_name='SVM',data_type='PCA')
pickle.dump(comp_model, open('/content/gdrive/MyDrive/ISS-Project/Models/final_svm_pca_model.pkl', 'wb'))
y_hat = comp_model.predict(features_test_pca)
np.save('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/final_svm_pca_predictions.npy', y_hat)
acc = metrics.accuracy_score(y_test, y_hat)
print(acc)

0.18862876254180602


In [ ]:
comp_model = train_model(features_train_lda, y_train, model_name='SVM',data_type='LDA')
pickle.dump(comp_model, open('/content/gdrive/MyDrive/ISS-Project/Models/final_svm_lda_model.pkl', 'wb'))
y_hat = comp_model.predict(features_test_lda)
np.save('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/final_svm_lda_predictions.npy', y_hat)
acc = metrics.accuracy_score(y_test, y_hat)
print(acc)


0.7743589743589744


In [ ]:
comp_model = train_model(lda_pca_train, y_train, model_name='SVM',data_type='LDA-PCA')
pickle.dump(comp_model, open('/content/gdrive/MyDrive/ISS-Project/Models/final_svm_lda_pca_model.pkl', 'wb'))
y_hat = comp_model.predict(lda_pca_test)
np.save('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/final_svm_lda_pca_predictions.npy', y_hat)
acc = metrics.accuracy_score(y_test, y_hat)
print(acc)

0.19175027870680045


### Model #3 KNN

In [ ]:
def KNNGrid(X_train, y_train):
    classifier = KNeighborsClassifier()
    parameters = {'n_neighbors': [5, 51, 101]}
    clf = GridSearchCV(estimator=classifier, param_grid=parameters, scoring='accuracy', n_jobs=-1, verbose=3,
                       refit=True)
    clf.fit(X_train, y_train)
    best_accuracy = clf.best_score_
    best_parameters = clf.best_params_
    print("Best Accuracy: {:.2f} %".format(best_accuracy * 100))
    print("Best Parameters:", best_parameters)
    return clf

In [ ]:
# KNN PCA
knn_pca_model = KNNGrid(features_train_pca, y_train)
pickle.dump(knn_pca_model, open('/content/gdrive/MyDrive/ISS-Project/Models/knn_pca_model.pkl', 'wb'))
#knn_pca_model = pickle.load(open('/content/gdrive/MyDrive/ISS-Project/Models/knn_pca_model.pkl', 'rb'))
print(knn_pca_model)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   53.6s finished


Best Accuracy: 98.89 %
Best Parameters: {'n_neighbors': 5}
GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=-1,
             param_grid={'n_neighbors': [5, 51, 101]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring='accuracy',
             verbose=3)


In [ ]:
# KNN LDA
knn_lda_model = KNNGrid(features_train_lda, y_train)
pickle.dump(knn_lda_model, open('/content/gdrive/MyDrive/ISS-Project/Models/knn_lda_model.pkl', 'wb'))
#knn_lda_model = pickle.load(open('/content/gdrive/MyDrive/ISS-Project/Models/knn_lda_model.pkl', 'rb'))
print(knn_lda_model)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   17.9s finished


Best Accuracy: 96.68 %
Best Parameters: {'n_neighbors': 51}
GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=-1,
             param_grid={'n_neighbors': [5, 51, 101]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring='accuracy',
             verbose=3)


In [ ]:
# KNN PCA-LDA
knn_lda_pca_model = KNNGrid(lda_pca_train, y_train)
pickle.dump(knn_lda_pca_model, open('/content/gdrive/MyDrive/ISS-Project/Models/knn_lda_pca_model.pkl', 'wb'))
#knn_lda_pca_model = pickle.load(open('/content/gdrive/MyDrive/ISS-Project/Models/knn_lda_pca_model.pkl', 'rb'))
print(knn_lda_pca_model)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    2.1s finished


Best Accuracy: 98.45 %
Best Parameters: {'n_neighbors': 5}
GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=-1,
             param_grid={'n_neighbors': [5, 51, 101]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring='accuracy',
             verbose=3)


#### Train Best Model ( KNN )

##### PCA

1.   Best Parameters: {'n_neighbors': 5}
2.   Best Accuracy: 98.89 %
3.   Best Accuracy Testing Data: 18 %


##### LDA
1.   Best Parameters: {'n_neighbors': 51}
2.   Best Accuracy: 96.68 %
3.   Best Accuracy Testing Data: 77 %

##### LDA - PCA
1.   Best Parameters: {'n_neighbors': 5}
2.   Best Accuracy: 98.45 %
3.   Best Accuracy Testing Data:  20%



In [ ]:
comp_model = train_model(features_train_pca, y_train, model_name='KNN',data_type='PCA')
pickle.dump(comp_model, open('/content/gdrive/MyDrive/ISS-Project/Models/final_knn_pca_model.pkl', 'wb'))
y_hat = comp_model.predict(features_test_pca)
np.save('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/final_knn_pca_predictions.npy', y_hat)
acc = metrics.accuracy_score(y_test, y_hat)
print(acc)

0.18573021181716834


In [ ]:
comp_model = train_model(features_train_lda, y_train, model_name='KNN',data_type='LDA')
pickle.dump(comp_model, open('/content/gdrive/MyDrive/ISS-Project/Models/final_knn_lda_model.pkl', 'wb'))
y_hat = comp_model.predict(features_test_lda)
np.save('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/final_knn_lda_predictions.npy', y_hat)
acc = metrics.accuracy_score(y_test, y_hat)
print(acc)


0.7745819397993311


In [ ]:
comp_model = train_model(lda_pca_train, y_train, model_name='KNN',data_type='LDA-PCA')
pickle.dump(comp_model, open('/content/gdrive/MyDrive/ISS-Project/Models/final_knn_lda_pca_model.pkl', 'wb'))
y_hat = comp_model.predict(lda_pca_test)
np.save('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/final_knn_lda_pca_predictions.npy', y_hat)
acc = metrics.accuracy_score(y_test, y_hat)
print(acc)

0.20780379041248606


### Model #4 Bagging

In [ ]:
def BaggingGrid(X_train, y_train):
    classifier = BaggingClassifier(random_state=0)
    parameters = {'n_estimators': [10, 20, 40]}
    clf = GridSearchCV(estimator=classifier, param_grid=parameters, scoring='accuracy', n_jobs=-1, verbose=3,
                       refit=True)
    clf.fit(X_train, y_train)
    best_accuracy = clf.best_score_
    best_parameters = clf.best_params_
    print("Best Accuracy: {:.2f} %".format(best_accuracy * 100))
    print("Best Parameters:", best_parameters)
    return clf

In [ ]:
# Bagging PCA
bag_pca_model = BaggingGrid(features_train_pca, y_train)
pickle.dump(bag_pca_model, open('/content/gdrive/MyDrive/ISS-Project/Models/bag_pca_model.pkl', 'wb'))
#bag_pca_model = pickle.load(open('/content/gdrive/MyDrive/ISS-Project/Models/bag_pca_model.pkl', 'rb'))
print(bag_pca_model)


Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 16.9min finished


Best Accuracy: 95.91 %
Best Parameters: {'n_estimators': 40}
GridSearchCV(cv=None, error_score=nan,
             estimator=BaggingClassifier(base_estimator=None, bootstrap=True,
                                         bootstrap_features=False,
                                         max_features=1.0, max_samples=1.0,
                                         n_estimators=10, n_jobs=None,
                                         oob_score=False, random_state=0,
                                         verbose=0, warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'n_estimators': [10, 20, 40]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring='accuracy',
             verbose=3)


In [ ]:
# Bagging LDA
bag_lda_model = BaggingGrid(features_train_lda, y_train)
pickle.dump(bag_lda_model, open('/content/gdrive/MyDrive/ISS-Project/Models/bag_lda_model.pkl', 'wb'))
#bag_lda_model = pickle.load(open('/content/gdrive/MyDrive/ISS-Project/Models/bag_lda_model.pkl', 'rb'))
print(bag_lda_model)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.1min finished


Best Accuracy: 91.23 %
Best Parameters: {'n_estimators': 40}
GridSearchCV(cv=None, error_score=nan,
             estimator=BaggingClassifier(base_estimator=None, bootstrap=True,
                                         bootstrap_features=False,
                                         max_features=1.0, max_samples=1.0,
                                         n_estimators=10, n_jobs=None,
                                         oob_score=False, random_state=0,
                                         verbose=0, warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'n_estimators': [10, 20, 40]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring='accuracy',
             verbose=3)


In [ ]:
# Bagging PCA-LDA
bag_lda_pca_model = BaggingGrid(lda_pca_train, y_train)
pickle.dump(bag_lda_pca_model, open('/content/gdrive/MyDrive/ISS-Project/Models/bag_lda_pca_model.pkl', 'wb'))
#bag_lda_pca_model = pickle.load(open('/content/gdrive/MyDrive/ISS-Project/Models/bag_lda_pca_model.pkl', 'rb'))
print(bag_lda_pca_model)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   15.7s finished


Best Accuracy: 97.46 %
Best Parameters: {'n_estimators': 20}
GridSearchCV(cv=None, error_score=nan,
             estimator=BaggingClassifier(base_estimator=None, bootstrap=True,
                                         bootstrap_features=False,
                                         max_features=1.0, max_samples=1.0,
                                         n_estimators=10, n_jobs=None,
                                         oob_score=False, random_state=0,
                                         verbose=0, warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'n_estimators': [10, 20, 40]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring='accuracy',
             verbose=3)


#### Train Best Model ( Bagging )

##### PCA

1.   Best Parameters: {'n_estimators': 40}
2.   Best Accuracy: 95.91 %
3.   Best Accuracy Testing Data: 23%

##### LDA
1.   Best Parameters: {'n_estimators': 40}
2.   Best Accuracy: 91.23 %
3.   Best Accuracy Testing Data: 63 %

##### LDA - PCA
1.   Best Parameters: {'n_estimators': 20}
2.   Best Accuracy: 97.46 %
3.   Best Accuracy Testing Data: 19 %



In [ ]:
comp_model = train_model(features_train_pca, y_train, model_name='BAG',data_type='PCA')
pickle.dump(comp_model, open('/content/gdrive/MyDrive/ISS-Project/Models/final_bag_pca_model.pkl', 'wb'))
y_hat = comp_model.predict(features_test_pca)
np.save('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/final_bag_pca_predictions.npy', y_hat)
acc = metrics.accuracy_score(y_test, y_hat)
print(acc)

0.23612040133779263


In [ ]:
comp_model = train_model(features_train_lda, y_train, model_name='BAG',data_type='LDA')
pickle.dump(comp_model, open('/content/gdrive/MyDrive/ISS-Project/Models/final_bag_lda_model.pkl', 'wb'))
y_hat = comp_model.predict(features_test_lda)
np.save('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/final_bag_lda_predictions.npy', y_hat)
acc = metrics.accuracy_score(y_test, y_hat)
print(acc)


0.6336677814938685


In [ ]:
comp_model = train_model(lda_pca_train, y_train, model_name='BAG',data_type='LDA-PCA')
pickle.dump(comp_model, open('/content/gdrive/MyDrive/ISS-Project/Models/final_bag_lda_pca_model.pkl', 'wb'))
y_hat = comp_model.predict(lda_pca_test)
np.save('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/final_bag_lda_pca_predictions.npy', y_hat)
acc = metrics.accuracy_score(y_test, y_hat)
print(acc)

0.19085841694537348


### Model #5 Decision Tree

In [ ]:
def DTGrid(X_train, y_train):
    classifier = DecisionTreeClassifier()
    parameters = {'criterion': ['gini', 'entropy'], 'max_depth': [5, 10, 15, 20]}
    clf = GridSearchCV(estimator=classifier, param_grid=parameters, scoring='accuracy', n_jobs=-1, verbose=3,
                       refit=True)
    clf.fit(X_train, y_train)
    best_accuracy = clf.best_score_
    best_parameters = clf.best_params_
    print("Best Accuracy: {:.2f} %".format(best_accuracy * 100))
    print("Best Parameters:", best_parameters)
    return clf

In [ ]:
# Decision Tree PCA
dt_pca_model = DTGrid(features_train_pca, y_train)
pickle.dump(dt_pca_model, open('/content/gdrive/MyDrive/ISS-Project/Models/dt_pca_model.pkl', 'wb'))
#dt_pca_model = pickle.load(open('/content/gdrive/MyDrive/ISS-Project/Models/dt_pca_model.pkl', 'rb'))
print(dt_pca_model)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  5.1min finished


Best Accuracy: 85.95 %
Best Parameters: {'criterion': 'entropy', 'max_depth': 20}
GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs

In [ ]:
# Decision Tree LDA
dt_lda_model = DTGrid(features_train_lda, y_train)
pickle.dump(dt_lda_model, open('/content/gdrive/MyDrive/ISS-Project/Models/dt_lda_model.pkl', 'wb'))
#dt_lda_model = pickle.load(open('/content/gdrive/MyDrive/ISS-Project/Models/dt_lda_model.pkl', 'rb'))
print(dt_lda_model)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   16.4s finished


Best Accuracy: 80.69 %
Best Parameters: {'criterion': 'entropy', 'max_depth': 15}
GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs

In [ ]:
# Decision Tree LDA-PCA
dt_lda_pca_model = DTGrid(lda_pca_train, y_train)
pickle.dump(dt_lda_pca_model, open('/content/gdrive/MyDrive/ISS-Project/Models/dt_lda_pca_model.pkl', 'wb'))
#dt_lda_pca_model = pickle.load(open('/content/gdrive/MyDrive/ISS-Project/Models/dt_lda_pca_model.pkl', 'rb'))
print(dt_lda_pca_model)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    3.9s finished


Best Accuracy: 96.04 %
Best Parameters: {'criterion': 'entropy', 'max_depth': 15}
GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs

#### Train Best Model ( Decision Tree )

##### PCA #      

1.   Best Parameters: {'criterion': 'entropy', 'max_depth': 20}
2.   Best Accuracy Training Data: 85.95 %
3.   Best Accuracy Testing Data :  19.0%


##### LDA
1.   Best Parameters: {'criterion': 'entropy', 'max_depth': 15}
2.   Best Accuracy Training Data: 80.69 %
3.   Best Accuracy Testing Data :  52.0%

##### LDA PCA
1.   Best Parameters: {'criterion': 'entropy', 'max_depth': 15}
2.   Best Accuracy Training Data: 96.04 %
3.   Best Accuracy Testing Data :  17.0%




In [ ]:
comp_model = train_model(features_train_pca, y_train, model_name='DT',data_type='PCA')
pickle.dump(comp_model, open('/content/gdrive/MyDrive/ISS-Project/Models/final_dt_pca_model.pkl', 'wb'))
y_hat = comp_model.predict(features_test_pca)
np.save('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/final_dt_pca_predictions.npy', y_hat)
acc = metrics.accuracy_score(y_test, y_hat)
print(acc)

0.19643255295429207


In [ ]:

comp_model = train_model(features_train_lda, y_train, model_name='DT',data_type='LDA')
pickle.dump(comp_model, open('/content/gdrive/MyDrive/ISS-Project/Models/final_dt_lda_model.pkl', 'wb'))
y_hat = comp_model.predict(features_test_lda)
np.save('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/final_dt_lda_predictions.npy', y_hat)
acc = metrics.accuracy_score(y_test, y_hat)
print(acc)

0.5239687848383501


In [ ]:
comp_model = train_model(lda_pca_train, y_train, model_name='DT',data_type='LDA-PCA')
pickle.dump(comp_model, open('/content/gdrive/MyDrive/ISS-Project/Models/final_dt_lda_pca_model.pkl', 'wb'))
y_hat = comp_model.predict(lda_pca_test)
np.save('/content/gdrive/MyDrive/ISS-Project/NumpyArrays/final_dt_lda_pca_predictions.npy', y_hat)
acc = metrics.accuracy_score(y_test, y_hat)
print(acc)

0.17391304347826086
